In [ ]:
import json
import datetime
import xml.etree.ElementTree as ET

# Configuración general del podcast
PODCAST_TITLE = "Homilías Papa León XIV"
PODCAST_LINK = "https://igles-ia.es/podcast"
PODCAST_DESCRIPTION = "Podcast con los discursos y homilías del Santo Padre íntegras. Leídos con locutor profesional gracias a la IA.."
PODCAST_LANGUAGE = "es"
PODCAST_IMAGE = "https://igles-ia.es/images/podcast-cover.jpg"  # portada general
PODCAST_AUTHOR = "igles-ia.es"

# Ruta de tu JSON exportado desde el DataFrame
JSON_FILE = "../summaries-local/2025-08-15/iglesia.json"

# Carpeta donde están los MP3
AUDIO_BASE_URL = "https://igles-ia.es/audio/"
# Carpeta donde están las imágenes por episodio
IMAGE_BASE_URL = "https://igles-ia.es/images/episodios/"

# Cargar datos del JSON
with open(JSON_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# Crear estructura RSS con espacio de nombres iTunes
ITUNES_NS = "http://www.itunes.com/dtds/podcast-1.0.dtd"
ET.register_namespace("itunes", ITUNES_NS)

rss = ET.Element("rss", {
    "version": "2.0",
    "xmlns:itunes": ITUNES_NS
})

channel = ET.SubElement(rss, "channel")

# Información general del podcast
ET.SubElement(channel, "title").text = PODCAST_TITLE
ET.SubElement(channel, "link").text = PODCAST_LINK
ET.SubElement(channel, "language").text = PODCAST_LANGUAGE
ET.SubElement(channel, "description").text = PODCAST_DESCRIPTION
ET.SubElement(channel, "itunes:author").text = PODCAST_AUTHOR
ET.SubElement(channel, "itunes:image", {"href": PODCAST_IMAGE})

# Añadir episodios
for idx in range(len(data["titulo"])):
    item = ET.SubElement(channel, "item")

    titulo = data["titulo"][str(idx)]
    fecha = data["fecha"][str(idx)]
    descripcion = data["texto"][str(idx)].split("\n")[0]  # primera línea como resumen
    filename = data["filename"][str(idx)] + ".mp3"

    # Enlaces
    audio_url = AUDIO_BASE_URL + filename
    imagen_url = IMAGE_BASE_URL + data["filename"][str(idx)] + ".jpg"

    # Campos del episodio
    ET.SubElement(item, "title").text = titulo
    ET.SubElement(item, "description").text = descripcion
    ET.SubElement(item, "pubDate").text = datetime.datetime.strptime(fecha, "%Y-%m-%d").strftime("%a, %d %b %Y 00:00:00 GMT")
    ET.SubElement(item, "enclosure", {
        "url": audio_url,
        "type": "audio/mpeg"
    })
    ET.SubElement(item, "itunes:image", {"href": imagen_url})

# Guardar archivo XML
tree = ET.ElementTree(rss)
tree.write("podcast.xml", encoding="utf-8", xml_declaration=True)

print("✅ RSS generado: podcast.xml")
